# 1. Random Forest

In [4]:
%pip install scikit-learn pandas

  Using cached scikit_learn-1.6.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached pandas-2.2.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.5-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.15.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.5.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached scikit_learn-1.6.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.2 MB)
Using cached pandas-2.2.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached joblib-1.5.0-py3-none-any.whl (307 kB)
Using cached numpy-2.2.5-cp313-cp313-manylinux_2_17_x86_64.manylin

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [7]:
data = pd.read_csv('../../data/processed/train_preprocess_v1.csv')
#data = pd.read_csv('../../data/processed/train_limpieza_v1.csv')
data.head(5)


,id,label,statement,subject,speaker,speaker_job,state_info,party_affiliation,party_affiliation_uni,party_affiliation_category_map,...,pos_info_without_stopwords,pos_freq_without_stopwords,lemma_freq_without_stopwords,tag_freq_without_stopwords,processed_subject,speaker_entities,speaker_type,speaker_job_tokens,state_info_tokens,party_affiliation_tokens
0,81f884c64a7,1,china is in the south china sea and (building)...,"china,foreign-policy,military",donald-trump,president-elect,new york,republican,republican,political-affiliation,...,"[{'lemma': 'china', 'pos': 'PROPN', 'tag': 'NN...","Counter({'PROPN': 4, 'NOUN': 4, 'ADJ': 1, 'VER...","Counter({'china': 2, 'south': 1, 'sea': 1, 'bu...","Counter({'NNP': 4, 'NN': 3, 'JJ': 1, 'NNS': 1,...","['china', 'foreign-policy', 'military']",['donald trump'],['PERSON'],"['president', '-', 'elect']","['new', 'york']",['republican']
1,30c2723a188,0,with the resources it takes to execute just ov...,health-care,chris-dodd,u.s. senator,connecticut,democrat,democrat,political-affiliation,...,"[{'lemma': 'resource', 'pos': 'NOUN', 'tag': '...","Counter({'NOUN': 7, 'VERB': 4, 'PROPN': 2, 'AD...","Counter({'resource': 1, 'take': 1, 'execute': ...","Counter({'NN': 4, 'NNS': 3, 'VB': 2, 'NNP': 2,...",['health-care'],['chris dodd'],['PERSON'],"['u.s', '.', 'senator']",['connecticut'],['democrat']
2,6936b216e5d,0,the (wisconsin) governor has proposed tax give...,"corporations,pundits,taxes,abc-news-week",donna-brazile,political commentator,"washington, d.c.",democrat,democrat,political-affiliation,...,"[{'lemma': 'wisconsin', 'pos': 'PROPN', 'tag':...","Counter({'NOUN': 4, 'PROPN': 1, 'VERB': 1})","Counter({'wisconsin': 1, 'governor': 1, 'propo...","Counter({'NN': 2, 'NNS': 2, 'NNP': 1, 'VBN': 1})","['corporations', 'pundits', 'taxes', 'abc-news...",['donna brazile'],['PERSON'],"['political', 'commentator']","['washington', ',', 'd.c', '.']",['democrat']
3,b5cd9195738,1,says her representation of an ex-boyfriend who...,"candidates-biography,children,ethics,families,...",rebecca-bradley,non-define,non-define,none,none,other-political-groups,...,"[{'lemma': 'say', 'pos': 'VERB', 'tag': 'VBZ',...","Counter({'NOUN': 9, 'VERB': 1, 'ADJ': 1})","Counter({'say': 1, 'representation': 1, 'ex': ...","Counter({'NN': 8, 'VBZ': 1, 'NNS': 1, 'JJ': 1})","['candidates-biography', 'children', 'ethics',...",['rebecca bradley'],['PERSON'],"['non', '-', 'define']","['non', '-', 'define']",['none']
4,84f8dac7737,0,at protests in wisconsin against proposed coll...,"health-care,labor,state-budget",republican-party-wisconsin,non-define,wisconsin,republican,republican,political-affiliation,...,"[{'lemma': 'protest', 'pos': 'NOUN', 'tag': 'N...","Counter({'NOUN': 7, 'VERB': 4, 'ADJ': 3})","Counter({'protest': 1, 'wisconsin': 1, 'propos...","Counter({'NNS': 4, 'NN': 3, 'JJ': 3, 'VBN': 2,...","['health-care', 'labor', 'state-budget']","['republican party', 'wisconsin']","['ORG', 'GPE']","['non', '-', 'define']",['wisconsin'],['republican']


In [78]:
# Seleccionar las características y etiquetas
X = data['statement']  # Texto de la afirmación
y = data['label']  # Etiqueta de veracidad (0 para falso, 1 para verdadero)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorización del texto utilizando TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [93]:

# Entrenamiento del modelo Random Forest
class_weights = {0: 5, 1: 2}  # No modificar los pesos, se empeora
rf_model = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',        # 'max_features': ['sqrt', 'log2', None],
    class_weight=class_weights, #'balanced
    random_state=42
)
rf_model.fit(X_train_tfidf, y_train)

# Realizar predicciones
y_pred = rf_model.predict(X_test_tfidf)




In [94]:
# Evaluar el modelo con Macro Average F1-Score
# Reporte completo de métricas
print(classification_report(y_test, y_pred))

macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")

              precision    recall  f1-score   support

           0       0.42      0.66      0.51       923
           1       0.74      0.52      0.61      1762

    accuracy                           0.57      2685
   macro avg       0.58      0.59      0.56      2685
weighted avg       0.63      0.57      0.58      2685

Macro Average F1-Score: 0.5604


In [55]:
import joblib
# Guardar el vectorizador TF-IDF
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')


['tfidf_vectorizer.joblib']

## Random forest solo con columnas determinadas

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Cargar el dataset
data = pd.read_csv('../../data/processed/train_preprocess_v1.csv')

# Seleccionar las características relevantes y la etiqueta
X = data[['statement', 'subject', 'party_affiliation', 'speaker', 'speaker_job', 'state_info', 'party_affiliation_uni']]  # Columnas relevantes
y = data['label']  # Etiqueta de veracidad

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorización del texto utilizando TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['statement'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['statement'])

# Entrenamiento del modelo Random Forest
class_weights = {0: 5, 1: 2}  # No modificar los pesos, ya que empeora el modelo
rf_model = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',        # 'max_features': ['sqrt', 'log2', None],
    class_weight=class_weights, #'balanced'
    random_state=42
)
rf_model.fit(X_train_tfidf, y_train)

# Realizar predicciones
y_pred = rf_model.predict(X_test_tfidf)

# Evaluar el modelo con Macro Average F1-Score
# Reporte completo de métricas
print(classification_report(y_test, y_pred))

macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")


              precision    recall  f1-score   support

           0       0.41      0.67      0.51       923
           1       0.74      0.50      0.60      1762

    accuracy                           0.56      2685
   macro avg       0.58      0.59      0.56      2685
weighted avg       0.63      0.56      0.57      2685

Macro Average F1-Score: 0.5557


### Aplico Smote

In [ ]:
%pip install imblearn SMOTE

  Using cached imblearn-0.0-py2.py3-none-any.whl.metadata (355 bytes)
  Using cached imbalanced_learn-0.13.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached sklearn_compat-0.1.3-py3-none-any.whl.metadata (18 kB)
Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
Using cached imbalanced_learn-0.13.0-py3-none-any.whl (238 kB)
Using cached sklearn_compat-0.1.3-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [imblearn]
Note: you may need to restart the kernel to use updated packages.


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from imblearn.over_sampling import SMOTE  # Importamos SMOTE

# Cargar el dataset
data = pd.read_csv('../../data/processed/train_preprocess_v1.csv')

# Seleccionar las características relevantes y la etiqueta
X = data[['statement', 'subject', 'party_affiliation', 'speaker', 'speaker_job', 'state_info', 'party_affiliation_uni']]  # Columnas relevantes
y = data['label']  # Etiqueta de veracidad

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorización del texto utilizando TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['statement'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['statement'])

# --- Aplicar SMOTE solo en el conjunto de entrenamiento ---
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

# Entrenamiento del modelo Random Forest
class_weights = {0: 5, 1: 2}  # No modificar los pesos, ya que empeora el modelo
rf_model = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',        # 'max_features': ['sqrt', 'log2', None],
    class_weight=class_weights, #'balanced'
    random_state=42
)
rf_model.fit(X_train_smote, y_train_smote)

# Realizar predicciones
y_pred = rf_model.predict(X_test_tfidf)

# Evaluar el modelo con Macro Average F1-Score
# Reporte completo de métricas
print(classification_report(y_test, y_pred))

macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")


              precision    recall  f1-score   support

           0       0.39      0.80      0.52       923
           1       0.76      0.33      0.46      1762

    accuracy                           0.49      2685
   macro avg       0.57      0.57      0.49      2685
weighted avg       0.63      0.49      0.48      2685

Macro Average F1-Score: 0.4915


### Aplico la reducción de la mayoritaria

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Cargar el dataset
data = pd.read_csv('../../data/processed/train_preprocess_v1.csv')

# Seleccionar las características relevantes y la etiqueta
X = data[['statement', 'subject', 'party_affiliation', 'speaker', 'speaker_job', 'state_info', 'party_affiliation_uni']]  # Columnas relevantes
y = data['label']  # Etiqueta de veracidad

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorización del texto utilizando TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['statement'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['statement'])

# --- Realizar submuestreo (undersampling) en el conjunto de entrenamiento ---
# Extraemos los DataFrames de cada clase
df_train = pd.DataFrame(X_train_tfidf.toarray())  # Convertimos a DataFrame para manipularlo
df_train['label'] = y_train

# Extraemos las clases 0 (minoritaria) y 1 (mayoritaria)
df_train_class_0 = df_train[df_train['label'] == 0]
df_train_class_1 = df_train[df_train['label'] == 1]

# Realizamos un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_train_class_0)
df_train_class_1_under = df_train_class_1.sample(n=n_minority, random_state=42)

# Combinamos ambas clases y mezclamos los registros
df_train_balanced = pd.concat([df_train_class_0, df_train_class_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)

# Volver a separar las características y etiquetas después del submuestreo
X_train_balanced = df_train_balanced.drop(columns=['label'])
y_train_balanced = df_train_balanced['label']

# Entrenamiento del modelo Random Forest
class_weights = {0: 5, 1: 2}  # No modificar los pesos, ya que empeora el modelo
rf_model = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',        # 'max_features': ['sqrt', 'log2', None],
    class_weight=class_weights, #'balanced'
    random_state=42
)
rf_model.fit(X_train_balanced, y_train_balanced)

# Realizar predicciones
y_pred = rf_model.predict(X_test_tfidf)

# Evaluar el modelo con Macro Average F1-Score
# Reporte completo de métricas
print(classification_report(y_test, y_pred))

macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")


              precision    recall  f1-score   support

           0       0.34      0.93      0.50       923
           1       0.67      0.07      0.13      1762

    accuracy                           0.37      2685
   macro avg       0.51      0.50      0.32      2685
weighted avg       0.56      0.37      0.26      2685

Macro Average F1-Score: 0.3178


## Conclusiones,
Si dejo los pesos 5:2, de 0.56 bajo a 0.52, sin embargo, si modifico los pesos a 2:1, donde aparantemente se produce una mejora, cuando lo subo a kaggle, me baja de un 0.57 a un 0.50

## GridSearchCV para random forest

In [23]:
import pandas as pd
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score
import joblib

# Cargar el dataset de entrenamiento
data = pd.read_csv('../../data/processed/train_preprocess_v1.csv')

# Seleccionar las características y etiquetas
X = data['statement']  # Texto de la afirmación
y = data['label']  # Etiqueta de veracidad (0 para falso, 1 para verdadero)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorización del texto utilizando TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Guardar el vectorizador para su uso posterior (si es necesario)
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')

# Definir el modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Definir los parámetros a probar para el modelo
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'max_features': ['sqrt', 'log2', None],
    'class_weight': [None, 'balanced'],
}

# Realizar GridSearchCV para encontrar los mejores parámetros
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

# Obtener el mejor modelo después de la búsqueda de hiperparámetros
best_rf_model = grid_search.best_estimator_

# Realizar predicciones con el mejor modelo
y_test_pred = best_rf_model.predict(X_test_tfidf)

# Evaluar el modelo con el Macro Average F1-Score
macro_f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Macro Average F1-Score: {macro_f1:.4f}")

# Reporte completo de métricas
print(classification_report(y_test, y_test_pred))

Macro Average F1-Score: 0.4070
              precision    recall  f1-score   support

           0       0.46      0.01      0.02       923
           1       0.66      0.99      0.79      1762

    accuracy                           0.66      2685
   macro avg       0.56      0.50      0.41      2685
weighted avg       0.59      0.66      0.53      2685



# 2. DistilBERT (fine-tuning)

In [3]:
%pip install transformers datasets torch scikit-learn huggingface_hub[hf_xet] transformers[torch]


  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached torch-2.7.0-cp313-cp313-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached scikit_learn-1.6.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pandas-2.2.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached nvidia_nccl_cu12-2.26.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.0 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached idna-3.10-py3-

In [5]:
%pip install pandas numpy matplotlib seaborn scikit-learn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 47.2 MB/s eta 0:00:00
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 50.0 MB/s eta 0:00:00
Using cached kiwisolver-1.4.8-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [seaborn]m7/8 [seaborn]ib]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import datetime
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

/home/angel/LBBYs_CH2/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cargar el dataset de entrenamiento
data = pd.read_csv('../../data/processed/train_preprocess_v1.csv')

# Seleccionar las características y etiquetas
X = data['statement']  # Texto de la afirmación
y = data['label']  # Etiqueta de veracidad (0 para falso, 1 para verdadero)

In [3]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- Tokenización con DistilBERT ---
# Cargar el tokenizador de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [4]:
# Tokenizar las afirmaciones de entrenamiento y prueba
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)

# Convertir a formato Dataset compatible con Hugging Face
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask'], 'label': y_train.tolist()})
test_dataset = Dataset.from_dict({'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask'], 'label': y_test.tolist()})


In [10]:
!pip install accelerate>=0.26.0


In [6]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Cargar el modelo DistilBERT para clasificación
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Configuración de los parámetros de entrenamiento
# training_args = TrainingArguments(
#     output_dir='./results',          # Carpeta para guardar los resultados
#     num_train_epochs=3,              # Número de épocas
#     per_device_train_batch_size=8,   # Tamaño del batch de entrenamiento
#     per_device_eval_batch_size=16,   # Tamaño del batch de evaluación
#     warmup_steps=500,                # Número de pasos para el calentamiento
#     weight_decay=0.01,               # Decaimiento de peso
#     logging_dir='./logs',            # Carpeta para los logs
#     logging_steps=10,                # Intervalo de logging
# )
training_args = TrainingArguments(
    output_dir='./results',          # Carpeta para guardar los resultados
    num_train_epochs=1,              # Usamos 1 época para prueba rápida
    per_device_train_batch_size=4,   # Tamaño de batch reducido
    per_device_eval_batch_size=8,    # Tamaño de batch reducido
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import numpy as np
from sklearn.metrics import f1_score
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    # p.predictions es un numpy.ndarray de forma (batch_size, num_labels)
    preds = np.argmax(p.predictions, axis=1)        # ahora preds es un array de 0s y 1s
    labels = p.label_ids                            # etiquetas verdaderas
    f1 = f1_score(labels, preds, average='macro')   # F1 macro
    return {'f1': f1}



# Iniciar el entrenamiento con Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)
# Entrenar el modelo
trainer.train()

/home/angel/LBBYs_CH2/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.656300
1000,0.633000
1500,0.630200


TrainOutput(global_step=1567, training_loss=0.6402191449368008, metrics={'train_runtime': 2140.8003, 'train_samples_per_second': 2.926, 'train_steps_per_second': 0.732, 'total_flos': 829908252579840.0, 'train_loss': 0.6402191449368008, 'epoch': 1.0})

In [8]:
# Evaluar (aquí compute_metrics ya usará numpy.argmax)
trainer.evaluate()

# Predecir sobre test_dataset
pred_out = trainer.predict(test_dataset)

# Sacar las predicciones
test_preds = np.argmax(pred_out.predictions, axis=1)

# Reporte completo de métricas
print(classification_report(y_test, test_preds))
print("Macro F1:", f1_score(y_test, test_preds, average='macro'))


/home/angel/LBBYs_CH2/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


/home/angel/LBBYs_CH2/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


              precision    recall  f1-score   support

           0       0.52      0.33      0.41       923
           1       0.71      0.84      0.77      1762

    accuracy                           0.67      2685
   macro avg       0.61      0.59      0.59      2685
weighted avg       0.64      0.67      0.64      2685

Macro F1: 0.5863277630992005


# 3. Entrega Kaggle

## Random Forest


In [ ]:
import pandas as pd
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Cargar el dataset de test
df_test = pd.read_csv('../../data/processed/test_preprocess_v1.csv')  # Asegúrate de usar la ruta correcta

# --- Preprocesamiento ---
# 1. Asegurarse de que el texto esté preprocesado de la misma manera que en el entrenamiento:
#    Esto incluye la vectorización TF-IDF.

# Cargar el vectorizador TF-IDF entrenado previamente
tfidf_vectorizer = joblib.load('tfidf_vectorizer.joblib')

# Aplicar la transformación de texto en los datos de test
X_test_tfidf = tfidf_vectorizer.transform(df_test['statement'])

# --- Predicción ---
# Cargar el modelo Random Forest entrenado
rf_model = joblib.load('random_forest_model.joblib')

# Realizar las predicciones en los datos de test
y_test_pred = rf_model.predict(X_test_tfidf)

# --- Exportar Submission ---
# Crear el archivo CSV con las columnas requeridas: 'id' y 'label'
submission = pd.DataFrame({
    'id': df_test['id'],  # Usar el 'id' del dataset de prueba
    'label': y_test_pred  # Las predicciones del modelo
})

# Obtener la fecha actual en formato 'YYYY-MM-DD'
current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Generar el nombre del archivo con la fecha actual
filename = f'randomforest_{current_date}.csv'

# Guardar el archivo con el nombre que incluye la fecha
submission.to_csv(filename, index=False)

print(f"Archivo de submission '{filename}' generado correctamente.")


Archivo de submission 'randomforest_2025-05-13_00-23-50.csv' generado correctamente.


## Disbert

In [ ]:
import pandas as pd
import datetime
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from sklearn.metrics import classification_report, f1_score
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import Trainer, TrainingArguments, EvalPrediction
import numpy as np

# Cargar el dataset de test desde Kaggle (ruta del archivo que hayas cargado)
df_test = pd.read_csv('../../data/processed/test_preprocess_v1.csv')  # Asegúrate de usar la ruta correcta

# --- Preprocesamiento ---
# Cargar el tokenizador DistilBERT directamente desde Hugging Face
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenizar las afirmaciones de test
test_encodings = tokenizer(list(df_test['statement']), truncation=True, padding=True, max_length=512)

# Convertir a formato Dataset compatible con Hugging Face
test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'label': df_test['label'].tolist()  # Asume que df_test tiene la columna 'label'
})

# Cargar el modelo DistilBERT desde Hugging Face (sin necesidad de archivos previamente guardados)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir='./results',  # Carpeta para guardar los resultados
    num_train_epochs=1,      # Usamos 1 época para prueba rápida
    per_device_train_batch_size=4,  # Tamaño de batch reducido
    per_device_eval_batch_size=8,   # Tamaño de batch reducido
)

# Función de evaluación (cálculo de F1-score)
def compute_metrics(p: EvalPrediction):
    predictions, labels = p
    preds = np.argmax(predictions, axis=1)  # Convertir predicciones a etiquetas
    f1 = f1_score(labels, preds, average='macro')  # F1-score (macro)
    return {'f1': f1}

# Iniciar el entrenamiento con Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=test_dataset,  # Usar dataset de test como ejemplo para la predicción
    eval_dataset=test_dataset,   # Igual para evaluación
    compute_metrics=compute_metrics
)

# Entrenar el modelo (aunque solo sea por 1 época)
trainer.train()

# Evaluar el modelo (esto calculará las métricas de evaluación automáticamente)
trainer.evaluate()

# Realizar predicciones sobre test_dataset
pred_out = trainer.predict(test_dataset)

# Convertir las predicciones a etiquetas (tensores de PyTorch a numpy arrays)
test_preds = np.argmax(pred_out.predictions, axis=1)

# Reporte de métricas
print(classification_report(df_test['label'], test_preds))
print("Macro F1:", f1_score(df_test['label'], test_preds, average='macro'))

# --- Exportar Submission ---
# Crear el archivo CSV con las columnas requeridas: 'id' y 'label'
submission = pd.DataFrame({
    'id': df_test['id'],  # Usar el 'id' del dataset de prueba
    'label': test_preds  # Las predicciones del modelo
})

# Obtener la fecha actual en formato 'YYYY-MM-DD'
current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Generar el nombre del archivo con la fecha actual
filename = f'distilbert_{current_date}.csv'

# Guardar el archivo con el nombre que incluye la fecha
submission.to_csv(filename, index=False)

print(f"Archivo de submission '{filename}' generado correctamente.")
